In [2]:
from typing import List, Tuple

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from scipy import sparse
from scipy.stats import norm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import OneHotEncoder, StandardScaler

% matplotlib inline

In [3]:
def decrease_cardinality(
    df: pd.DataFrame,
    feature: str,
    threshold: float = 0.95,
) -> pd.DataFrame:
    """Уменьшает размерность признака по заданным параметрам"""
    x = df[feature].value_counts(1).cumsum()
    values = x[x >= threshold].index
    df.loc[df[feature].isin(values), feature] = values.min()
    
    return df

In [4]:
def feature_engineering(df: pd.DataFrame) -> pd.DataFrame:
    
    # на мобильных устройствах и ПК удобство просмотра одних и тех же зон может быть разным
    df['zone_id_os_id'] = df['zone_id'].astype('str') + '_' + df['os_id'].astype('str')
    
    # возможно, что некоторые баннеры показываются только 
    # в определнных местах и из-за этого плохие клики
    df['banner_id_zone_id'] = df['banner_id'].astype('str') + '_' + df['zone_id'].astype('str')

    # в разных странах может быть разная реакция на один и тот же баннер
    df['banner_id_country_id'] = df['banner_id'].astype('str') + '_' + df['country_id'].astype('str')
    
    return df

def create_and_fit_model(x_train: sparse, y_train: pd.Series, C=1):
    """Инитит обучает логрег модель."""
    lr = LogisticRegression(solver='liblinear', C=C)
    return lr.fit(x_train, y_train.values)

def cv(
    x_train: sparse,
    y_train: pd.Series,
    indexes: List[Tuple[int]],
    C=1,
    is_sliding=True
): 
    """
    Производит кроссвалидацию по времени. Возможны два режима работы: скользящим окном
    или расширающимся окном. В скользящем окне обучение происходит за промежуток времени,
    который каждый раз смещается вперед во времени. В расширяющемся окне данные для обучения
    все время увеличиваются вперед во времени.
    
    @param: x_train Спарс матрица с тренировочными данными
    @param: y_train pd.Series с таргетами
    @param: indexes Индексы для создания трейна и валидации
    @param: C обратный коэффициент регуляриации 
    @param: is_sliding использовать сколзящее окно или нет.
    
    """
    losses = []
    models = []
    
    # Стартовый индекс для расширяющегося окна
    train_start_index = indexes[0][0]
    
    for train_ind_start, train_ind_end, val_ind_end in indexes:
        if is_sliding:
            x_train_inner = x_train[train_ind_start: train_ind_end]
            y_train_inner = y_train[train_ind_start: train_ind_end]

            x_val_inner = x_train[train_ind_end: val_ind_end]
            y_val_inner = y_train[train_ind_end: val_ind_end]
        else:
            x_train_inner = x_train[train_start_index: train_ind_end]
            y_train_inner = y_train[train_start_index: train_ind_end]

            x_val_inner = x_train[train_ind_end: val_ind_end]
            y_val_inner = y_train[train_ind_end: val_ind_end]
        
        print(f'Train size: {x_train_inner.shape[0]}')
        print(f'Valid size: {x_val_inner.shape[0]}')
        
        model = create_and_fit_model(x_train_inner, y_train_inner, C=C)
        predict = model.predict_proba(x_val_inner)[:, 1]
        
        loss = log_loss(y_val_inner, predict)
        print(f'Logloss: {round(loss, 4)}')
        
        losses.append(loss)
        models.append(model)
         
    return losses, models

In [5]:
df = pd.read_csv(
    '../hw3/data/data.csv',
    usecols=[     
        'date_time',
        'zone_id',
        'banner_id',
        'os_id',
        'country_id',
        'clicks',
        'banner_id0',
        'coeff_sum0',
        'g0',
        'banner_id1',
        'coeff_sum1',
        'g1',
    ],
    dtype={
        'zone_id': np.uint16, 
        'banner_id': np.uint16,
        'campaign_clicks': np.uint16,
        'os_id': np.uint8,
        'country_id': np.uint8,
        'clicks': np.uint8,
    }
)

df = df.sort_values('date_time')
df = df[1:].reset_index(drop=True)
df['date_time'] = pd.to_datetime(df['date_time'])

df = decrease_cardinality(df, 'zone_id')
df = decrease_cardinality(df, 'banner_id')

train_index_max = df[df['date_time'] < '2021-10-02'].index.max()
test_index_max = df.shape[0]

In [6]:
temp_df = df[train_index_max:].copy()
temp_df['banner_id'] = temp_df['banner_id1']

df = pd.concat([df, temp_df], axis=0, ignore_index=True)

In [7]:
%%time
df = feature_engineering(df)

Wall time: 1min 20s


In [8]:
one_encoder = OneHotEncoder()
scaller = StandardScaler()

In [9]:
dates = pd.date_range('2021-09-26', '2021-10-02', freq='1d')
indexes = [df[df['date_time'] >= date].index.min() for date in dates]
indexes = tuple(zip(indexes[:-2], indexes[1:-1], indexes[2:]))

In [10]:
data = sparse.hstack([
    one_encoder.fit_transform(df[['banner_id_country_id']]),
    one_encoder.fit_transform(df[['banner_id_zone_id']]),
    one_encoder.fit_transform(df[['banner_id']]),
    one_encoder.fit_transform(df[['zone_id_os_id']]),
    one_encoder.fit_transform(df[['zone_id']]),
    one_encoder.fit_transform(df[['os_id']]),
    one_encoder.fit_transform(df[['country_id']]),
])

data = data.tocsr()

x_train, x_test = data[:train_index_max], data[train_index_max:]
y_train, y_test = df.iloc[:train_index_max]['clicks'], df.iloc[train_index_max:]['clicks']

In [11]:
losses, models = cv(x_train, y_train, indexes, C=0.16681)

Train size: 3102610
Valid size: 2367303
Logloss: 0.0812
Train size: 2367303
Valid size: 2307355
Logloss: 0.088
Train size: 2307355
Valid size: 2420588
Logloss: 0.1143
Train size: 2420588
Valid size: 1851189
Logloss: 0.1357
Train size: 1851189
Valid size: 1643447
Logloss: 0.1491


In [12]:
predict = [model.predict_proba(x_test)[:, 1] for model in models]

In [69]:
last_day_df = df.iloc[train_index_max:test_index_max][[
    'banner_id', 'banner_id0', 'g0', 'coeff_sum0', 'banner_id1', 'g1', 'coeff_sum1', 'clicks',
]]
last_day_df['banner_id0_proba'] = predict[-1][:test_index_max-train_index_max]
last_day_df['banner_id1_proba'] = predict[-1][test_index_max-train_index_max:]

# почему-то есть несколько строк с g1 < 0
last_day_df = last_day_df[last_day_df['g1'] >= 0].reset_index(drop=True)

last_day_df['coeff_sum0_new'] = np.log(last_day_df['banner_id0_proba']/(1-last_day_df['banner_id0_proba']))
last_day_df['coeff_sum1_new'] = np.log(last_day_df['banner_id1_proba']/(1-last_day_df['banner_id1_proba']))

last_day_df['N0'] = np.random.normal(last_day_df['coeff_sum0'], last_day_df['g0']**2)
last_day_df['N1'] = np.random.normal(last_day_df['coeff_sum1'], last_day_df['g1']**2)
last_day_df['p0'] = 1 - norm.cdf(
    (- last_day_df['coeff_sum0'] + last_day_df['coeff_sum1'])/(np.abs(last_day_df['g0']**2 + last_day_df['g1']**2)**0.5)  
)

last_day_df['N0_new'] = np.random.normal(last_day_df['coeff_sum0_new'], last_day_df['g0']**2)
last_day_df['N1_new'] = np.random.normal(last_day_df['coeff_sum1_new'], last_day_df['g1']**2)
last_day_df['p0_new'] = 1 - norm.cdf(
    (- last_day_df['coeff_sum0_new'] + last_day_df['coeff_sum1_new'])/(np.abs(last_day_df['g0']**2 + last_day_df['g1']**2)**0.5)
)

last_day_df['ratio'] = last_day_df['p0_new']/last_day_df['p0']

last_day_df['lambda'] = 10

# рассматриваем только те случаи в которых был показан banner_id0
cond = (last_day_df['banner_id'] == last_day_df['banner_id0'])

(last_day_df[cond]['clicks']*last_day_df[cond][['ratio', 'lambda']].min(axis=1)).sum()/last_day_df[cond].shape[0]

0.07803494335292893

In [51]:
last_day_df['clicks'].mean()

0.035221482500836836

Задавал вопрос в чате на тему интерпретируемости cips и получил ответ, что стоит относится к нему к ctr, который бы у нас получился если бы мы действовали новым полиси в тот момент когда показывали баннеры, на которых был посчитан cips.

По результатам работы получил cips 0.78, что меня насторожило. В реальной жизни ctr 7.8% на общей выборке показов баннеров кажется чем-то высоким. Согласен, что отдельные баннеры в особенных кампаниях возможно и могут его достичь, но для среднего по больнице это много. Все дело в лямбде и в формуле. Мы считаем отношение вероятностей (вероятность того, что баннер 0 будет кликнут при показе баннера 0 и 1) новой и старой полиси вот только тут опять возникают вопросы:
<li>Два ли баннера соревнуются в показе между собой или на самом деле их намного больше?</li>
<li>Почему в новой полиси мы по прежднему сравниваем баннер 0 и баннер 1? Ведь в новой полиси возможно имеет смысл сравнивать баннер 0 с баннером 100 или с банннером 200 т.к. у них будет выше вероятность показа чем у баннера 1.</li>
<li>Каким образом происходят показы если вероятность показать баннер 0 чуть больше чем вероятность показать баннер 1? Если мы всегда показываем тот баннер чья вероятность выше, то совершенно неправильно оценивать ctr по cips. Во сколько бы раз вероятность новой полиси не была больше вероятности старой бОльшую награду мы с этого не получим</li>
<li>Параметр лямбда вообще может и не быть равен 10</li>